In [1]:
import sklearn
import numpy as np
from scipy import io
import tensorflow as tf
import os
import time
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ALL = h5py.File('/Users/ssprl/Desktop/MachineryFIO_0features.mat')


In [3]:
data =  np.transpose(ALL['trainingData'].value, axes=(2,1,0))

labels =  np.transpose(ALL['trainingLabels'].value)

print("Data Shape ", data.shape)
print("Labels Shape", labels.shape)

Data Shape  (1408020, 169, 11)
Labels Shape (1408020, 169)


In [5]:
def init_weights(shape):
    init_random_dist=tf.truncated_normal(shape,stddev=0.05)
    return tf.Variable(init_random_dist)


In [6]:
def init_bias(shape):
    init_bias_vals=tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [7]:
def conv2D(x,W):
    return tf.nn.conv2d(x,W,strides=[1,2,2,1],padding='SAME')

In [8]:
def max_pool_3by3(x):
    
    return tf.nn.max_pool(x,ksize=[1,3,3,1],strides=[1,3,3,1],padding='SAME')


In [9]:
def convolutional_layer(input_x,shape):
    W=init_weights(shape)
    b=init_bias([shape[52]])
    return tf.nn.relu(conv2D(input_x,W)+b)



In [10]:
def normal_full_layer(input_layer,size):
    input_size=int(input_layer.get_shape()[1])
    W=init_weights([input_size,size])
    b=init_bias([size])
    return tf.matmul(input_layer,W)+b

In [11]:
#with tf.name_scope("inputs"):
x = tf.placeholder(tf.float32,[None, 129, 11],name = "x-input")    
x_image = tf.reshape(x, [-1, 129, 11, 1], "x-image")    
y_true = tf.placeholder(tf.float32, [None, 129], name = "y-input")    
#y_true.shape

In [12]:
#with tf.name_scope("hyperparameters"):
learning_rate = tf.placeholder(tf.float32, name = "learning_rate")

In [13]:
keep_prob = tf.placeholder(tf.float32)

In [14]:
#with tf.name_scope("model"):

W1 = tf.Variable(tf.truncated_normal([5,1,1,52], stddev=0.05))
B1 = tf.Variable(tf.ones([52])/10)
Y1 = tf.nn.relu(tf.nn.conv2d(x_image, W1, strides=[1,1,1,1], padding='SAME') + B1)

P1 = max_pool_3by3(Y1)    
P1.shape

W2 = tf.Variable(tf.truncated_normal([5,1,52,78], stddev=0.05))
B2 = tf.Variable(tf.ones([78])/10)
Y2 = tf.nn.relu(tf.nn.conv2d(P1, W2, strides=[1,1,1,1], padding='SAME') + B2) 
Y2.shape

convo_2_flat = tf.reshape(Y2,[-1,43*4*78])

full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))
full_layer_one.shape

full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob)

full_layer_two = tf.nn.relu(normal_full_layer(full_layer_one,1024))
full_layer_two.shape

full_two_dropout = tf.nn.dropout(full_layer_two,keep_prob)

linear_W = tf.Variable(tf.truncated_normal([1024,129], stddev=0.05))
linear_b = tf.Variable(tf.ones([129])/10)
y_pred = tf.matmul(full_layer_two, linear_W) + linear_b
#y_pred=tf.reshape(y_pred, [1, 129])
y_pred = tf.identity(y_pred, name="output_t")


In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    variable_parameters = 1
    for dim in variable.get_shape():
        variable_parameters *= dim.value
    total_parameters += variable_parameters

print("Total number of trainable parameters: %d" % total_parameters)

In [ ]:
Y2.shape



In [15]:
#with tf.name_scope("loss"):
cross_entropy = tf.reduce_mean(tf.squared_difference(y_pred,y_true), name = "loss")
    #is_correct = tf.equal(tf.argmax(y_true,1), tf.argmax(y_pred,1))
    #accuracy= sklearn.metrics.r2_score(y_true,y_pred)

In [16]:
#with tf.name_scope("train"):
# optimizer = tf.train.AdamOptimizer(learning_rate)
# train_op  = optimizer.minimize(cross_entropy) 
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy, name='my_training_step')


In [17]:
trainBatchSize=500
nDataSamples = len(data)
nDataSamples = 5000

In [18]:
checkpoint_dir = "CNN_2_onlyLPS/"
tf.gfile.MakeDirs(checkpoint_dir)



learningRates = np.hstack((1e-3*np.ones(6),
                           1e-4*np.ones(4),
                           1e-5*np.ones(2)))

num_epochs = len(learningRates)-11
with tf.Session() as sess:
    
    saver = tf.train.Saver()
    #save_relative_paths=True
    tf.train.write_graph(sess.graph_def,
                         checkpoint_dir,
                         "graph.pbtxt",
                         True)
    init = tf.global_variables_initializer()

    sess.run(init)
    
    for epoch in np.arange(num_epochs):
        idx = np.arange(len(labels))
        np.random.shuffle(idx)    

        #print("inside epoch loop = ", epoch)
        for i in np.arange(0, nDataSamples-1, trainBatchSize):
            
            x_batch = data[idx[i:i+trainBatchSize], 0:129, :]
            y_batch = labels[idx[i:i+trainBatchSize],0:129]

            feed = {x: x_batch, y_true: y_batch, learning_rate: learningRates[epoch], keep_prob: 0.75}
            
            sess.run(train_op,feed_dict=feed)

            if i%50 == 0:
                feed = {x: x_batch,y_true: y_batch, learning_rate: learningRates[epoch], keep_prob: 1.0}
                loss_value,prediction  = sess.run([cross_entropy,y_pred], feed_dict=feed)
                rmse = mean_squared_error(y_batch[:,0:129], prediction[:,0:129])**0.5
                mae = mean_absolute_error(y_batch[:,0:129],prediction[:,0:129])
                print("epoch: %2d step: %6d RMSE: %3.2f MAE: %3.2f loss: %6.4f" % \
                     (epoch, i, rmse, mae, loss_value))

    tf.gfile.MakeDirs(checkpoint_dir + '/model' + str(epoch))     
    checkpoint_file = os.path.join(checkpoint_dir + '/model' + str(epoch), "model")
    saver.save(sess, checkpoint_file)
    print("**** SAVED MODEL ****")      
    print("**** COMPLETED ALL THE EPOCHS ****")

epoch:  0 step:      0 RMSE: 14.95 MAE: 11.97 loss: 223.4496
epoch:  0 step:    500 RMSE: 13.39 MAE: 11.16 loss: 179.2961
epoch:  0 step:   1000 RMSE: 11.78 MAE: 9.47 loss: 138.8168
epoch:  0 step:   1500 RMSE: 12.42 MAE: 9.94 loss: 154.3161
epoch:  0 step:   2000 RMSE: 11.99 MAE: 9.80 loss: 143.6702
epoch:  0 step:   2500 RMSE: 11.69 MAE: 9.69 loss: 136.6727
epoch:  0 step:   3000 RMSE: 11.57 MAE: 9.62 loss: 133.9187
epoch:  0 step:   3500 RMSE: 11.31 MAE: 9.31 loss: 127.9972
epoch:  0 step:   4000 RMSE: 10.98 MAE: 9.01 loss: 120.6344
epoch:  0 step:   4500 RMSE: 11.71 MAE: 9.84 loss: 137.1127
**** SAVED MODEL ****
**** COMPLETED ALL THE EPOCHS ****


In [ ]:
y_batch.shape

In [ ]:
Y.shape